In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image 
import os
import random
import cv2

import tensorflow as tf
import keras 
from sklearn.model_selection import train_test_split 
from tensorflow.keras import utils
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.callbacks import ModelCheckpoint
from keras.layers import Flatten, BatchNormalization, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
read_file = pd.read_excel(r'C:\Users\anndz\Desktop\BP\COVID.metadata.xlsx')
read_file.to_csv(r'C:\Users\anndz\Desktop\BP\COVID.metadata.csv', index = None, header=True)
COVID = read_file

read_file = pd.read_excel(r'C:\Users\anndz\Desktop\BP\Normal.metadata.xlsx')
read_file.to_csv(r'C:\Users\anndz\Desktop\BP\Normal.metadata.csv', index = None, header=True)
Normal = read_file

read_file = pd.read_excel(r'C:\Users\anndz\Desktop\BP\Lung_Opacity.metadata.csv')
read_file.to_csv(r'C:\Users\anndz\Desktop\BP\Lung_Opacity.metadata.csv', index = None, header=True)
Lung_opacity = read_file

read_file = pd.read_excel(r'C:\Users\anndz\Desktop\BP\Viral Pneumonia.metadata.xlsx')
read_file.to_csv(r'C:\Users\anndz\Desktop\BP\Viral Pneumonia.metadata.csv', index = None, header=True)
Viral_Pneumonia = read_file

read_file = pd.read_excel(r'C:\Users\anndz\Desktop\BP\Tuberculosis.metadata.xlsx')
read_file.to_csv(r'C:\Users\anndz\Desktop\BP\Tuberculosis.metadata.csv', index = None, header=True)
Tuberculosis = read_file

Analyza

In [ ]:
Normal = pd.read_csv("Normal.metadata.csv")
COVID = pd.read_csv("COVID.metadata.csv")
Lung_opacity = pd.read_csv("Lung_Opacity.metadata.csv")
Pneumonia = pd.read_csv("Viral Pneumonia.metadata.csv")
Tuberculosis = pd.read_csv("Tuberculosis.metadata.csv")

Normal['Diagnoza'] = '0'
Normal['COVID'] = '0'
COVID['Diagnoza'] = '4'
COVID['COVID'] = '1'
Lung_opacity['Diagnoza'] = '1'
Lung_opacity['COVID'] = '0'
Pneumonia['Diagnoza'] = '2'
Pneumonia['COVID'] = '0'
Tuberculosis['Diagnoza'] = '3'
Tuberculosis['COVID'] = '0'
Tuberculosis['SIZE'] = '256*256'

data = pd.concat([Normal,COVID,Lung_opacity,Pneumonia,Tuberculosis])
data['Diagnoza']= data['Diagnoza'].astype(object)
data['COVID']= data['COVID'].astype(object)

In [ ]:
V = data['Diagnoza'].value_counts().index.tolist()
VC = data['Diagnoza'].value_counts()
sns.barplot(x= V, y = VC)

In [ ]:
V = data['COVID'].value_counts().index.tolist()
VC = data['COVID'].value_counts()
sns.barplot(x= V, y = VC)

Rozdelenie dát na trénovaciu a testovaciu množinu

In [ ]:
def read_data(folder,label):
    x = 0
    data = []
    path = os.path.join(folder, label)
    for images in os.listdir(path):
        img = cv2.imread(os.path.join(folder,label, images))[...,::-1]
        data.append(img)
    return data

In [ ]:
XCOV = read_data('','COVID')
xCOV_train, xCOV_test = train_test_split(XCOV, test_size=0.3, random_state=1)

for i in range(len(xCOV_test)):
    cv2.imwrite(r"C:\Users\anndz\Desktop\BP\COVID_test\COVID_test_%d.png" % i, xCOV_test[i])

for k in range(len(xCOV_train)):
    cv2.imwrite(r"C:\Users\anndz\Desktop\BP\COVID_train\COVID_train_%d.png" % k, xCOV_train[k])

In [ ]:
X_LO = read_data('','Lung_Opacity')
xLO_train, xLO_test = train_test_split(X_LO, test_size=0.3, random_state=1)

for i in range(len(xLO_test)):
    cv2.imwrite(r"C:\Users\anndz\Desktop\BP\Lung_Opacity_test\Lung_Opacity_test_%d.png" % i, xLO_test[i])

for k in range(len(xLO_train)):
    cv2.imwrite(r"C:\Users\anndz\Desktop\BP\Lung_Opacity_train\Lung_Opacity_train_%d.png" % k, xLO_train[k])

In [ ]:
X_VP = read_data('','Viral_Pneumonia')
xVP_train, xVP_test = train_test_split(X_VP, test_size=0.3, random_state=1)

for i in range(len(xVP_test)):
    cv2.imwrite(r"C:\Users\anndz\Desktop\BP\Viral_Pneumonia_test\Viral_Pneumonia_test_%d.png" % i, xVP_test[i])

for k in range(len(xVP_train)):
    cv2.imwrite(r"C:\Users\anndz\Desktop\BP\Viral_Pneumonia_train\Viral_Pneumonia_train_%d.png" % k, xVP_train[k])

In [ ]:
X_TB = read_data('','Tuberculosis')
xTB_train, xTB_test = train_test_split(X_TB, test_size=0.3, random_state=1)

for i in range(len(xTB_test)):
    cv2.imwrite(r"C:\Users\anndz\Desktop\BP\Tuberculosis_test\Tuberculosis_test_%d.png" % i, xTB_test[i])

for k in range(len(xTB_train)):
    cv2.imwrite(r"C:\Users\anndz\Desktop\BP\Tuberculosis_train\Tuberculosis_train_%d.png" % k, xTB_train[k])

In [ ]:
X_Normal = read_data('','Normal')
xNormal_train, xNormal_test = train_test_split(X_Normal, test_size=0.3, random_state=1)

for i in range(len(xNormal_test)):
    cv2.imwrite(r"C:\Users\anndz\Desktop\BP\Normal_test\Normal_test_%d.png" % i, xNormal_test[i])

for k in range(len(xNormal_train)):
    cv2.imwrite(r"C:\Users\anndz\Desktop\BP\Normal_train\Normal_train_%d.png" % k, xNormal_train[k])

Augumentácia dát

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=25,
        height_shift_range=0.1,
        width_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest')
i = 0
for batch in datagen.flow_from_directory(r'C:\Users\anndz\Desktop\BP\COVID_resized', batch_size=1024,target_size=(256,256),
                          save_to_dir=r'C:\Users\anndz\Desktop\BP\COVID_aug', save_format='png'):
    i += 1
    if i > 7:
        break

In [ ]:
i = 0
for batch in datagen.flow_from_directory(r'C:\Users\anndz\Desktop\BP\Tuberculosis_resized', batch_size=490,target_size=(256,256),
                          save_to_dir=r'C:\Users\anndz\Desktop\BP\Tuberculosis_aug', save_format='png'):
    i += 1
    if i > 14:
        break

In [ ]:
i = 0
for batch in datagen.flow_from_directory(r'C:\Users\anndz\Desktop\BP\Viral_Pneumonia_resized', batch_size=941,target_size=(256,256),
                          save_to_dir=r'C:\Users\anndz\Desktop\BP\Viral_Pneumonia_aug', save_format='png'):
    i += 1
    if i > 8:
        break

In [ ]:
i = 0
for batch in datagen.flow_from_directory(r'C:\Users\anndz\Desktop\BP\Lung_Opacity_resized', batch_size=1024,target_size=(256,256),
                          save_to_dir=r'C:\Users\anndz\Desktop\BP\Lung_Opacity_aug', save_format='png'):
    i += 1
    if i > 3:
        break

Predspracovanie

In [ ]:
Ycat_train = []
Ybin_train = []
train_labels = ['Normal_train', 'Lung_Opacity_train','Viral_Pneumonia_train','Tuberculosis_train','COVID_resized']
#train_labels = ['Normal_train', 'Lung_Opacity_resized','Viral_Pneumonia_resized','Tuberculosis_resized','COVID_resized']
#normal = 0, LO = 1, VP = 2, Tuberculosis = 3, COVID = 4
size = 256

def get_trainData(folder, Ycat_train, Ybin_train):
    x = 0
    data = []
    for label in train_labels: 
        path = os.path.join(folder, label)
        for images in os.listdir(path):
            img = cv2.imread(os.path.join(folder,label, images))[...,::-1]
            img = cv2.resize(img, (size, size))
            img = img.astype('float16')
            img /= 255
            data.append(img)
            category = np.zeros(5)
            category[x] = 1
            Ycat_train.append(category)
            if (label == 'COVID_resized'):
                Ybin_train.append(1)
            else:
                Ybin_train.append(0)
        x = x + 1
    return data

In [ ]:
Xdata_train = get_trainData('',Ycat_train, Ybin_train)
random.seed(151)
Xdata_train = random.sample(Xdata_train,len(Xdata_train))
random.seed(151)
Ycat_train=random.sample(Ycat_train,len(Ycat_train))
random.seed(151)
Ybin_train=random.sample(Ybin_train,len(Ybin_train))
X_train = np.array(Xdata_train)
Ycat_train = np.array(Ycat_train)
Ybin_train = np.array(Ybin_train)

In [ ]:
Ycat_test = []
Ybin_test = []
test_labels = ['Normal_test', 'Lung_Opacity_test','Viral_Pneumonia_test','Tuberculosis_test','COVID_test']
#normal = 0, LO = 1, VP = 2, Tuberculosis = 3, COVID = 4
size = 256

def get_testData(folder, Ycat_test, Ybin_test):
    x = 0
    data = []
    for label in test_labels: 
        path = os.path.join(folder, label)
        for images in os.listdir(path):
            img = cv2.imread(os.path.join(folder,label, images))[...,::-1]
            img = cv2.resize(img, (size, size))
            img = img.astype('float16')
            img /= 255
            data.append(img)
            category = np.zeros(5)
            category[x] = 1
            Ycat_test.append(category)
            if label == 'COVID_test':
                Ybin_test.append(1)
            else:
                Ybin_test.append(0)
        x = x + 1
    return data

In [ ]:
Xdata_test = get_testData('',Ycat_test, Ybin_test)
random.seed(151)
Xdata_test = random.sample(Xdata_test,len(Xdata_test))
random.seed(151)
Ycat_test=random.sample(Ycat_test,len(Ycat_test))
random.seed(151)
Ybin_test=random.sample(Ybin_test,len(Ybin_test))
X_test = np.array(Xdata_test)
Ycat_test = np.array(Ycat_test)
Ybin_test = np.array(Ybin_test)

Binárna klasifikácia

In [ ]:
modelBinary = Sequential()

modelBinary.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(256,256,3)))
modelBinary.add(MaxPool2D())

modelBinary.add(Conv2D(64,3,padding="same", activation="relu", input_shape=(256,256,3)))
modelBinary.add(MaxPool2D())

modelBinary.add(Conv2D(64,3,padding="same", activation="relu", input_shape=(256,256,3)))
modelBinary.add(MaxPool2D())

modelBinary.add(Dropout(0.2))

modelBinary.add(Flatten())
modelBinary.add(Dense(128,activation="relu"))
modelBinary.add(Dense(1, activation="sigmoid"))

modelBinary.summary() 

In [ ]:
adam = Adam()
modelBinary.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = modelBinary.fit(X_train, Ybin_train,epochs=4, validation_data=(X_test, Ybin_test))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(4)

plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predictions = model.predict(X_test)
pred = [int(round(x[0])) for x in predictions]

In [ ]:
print("Accuracy score:",round(accuracy_score(Ybin_test, pred),4))
print("Number of correctly classified samples:",accuracy_score(Ybin_test, pred, normalize=False))
print(classification_report(Ybin_test, pred, target_names = ['Non-COVID (Class 0)','COVID (Class 1)']))

In [ ]:
print('Confusion matrix : \n')
print(confusion_matrix(Ybin_test, pred)) #(actual, predicted)
ax = sns.heatmap(confusion_matrix(Ybin_test, pred), annot = True, fmt = 'g' ,vmin = 0, vmax = 2000,cmap = 'Blues')

Categorical

In [ ]:
modelCat = Sequential()

modelCat.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(256,256,3)))
modelCat.add(MaxPool2D())

modelCat.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(256,256,3)))
modelCat.add(MaxPool2D())

modelCat.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(256,256,3)))
modelCat.add(MaxPool2D())

modelCat.add(Dropout(0.2))

modelCat.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(256,256,3)))
modelCat.add(MaxPool2D())

modelCat.add(Flatten())
modelCat.add(Dense(128, activation='relu'))
modelCat.add(Dense(5, activation='softmax'))

modelCat.summary() 

In [ ]:
adam = Adam()
modelCat.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
f = modelCat.fit(X_train, Ycat_train, epochs = 4, validation_data=(X_test, Ycat_test))

In [ ]:
acc = f.history['accuracy']
val_acc = f.history['val_accuracy']
loss = f.history['loss']
val_loss = f.history['val_loss']

epochs_range = range(4)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predictions = modelCat.predict(X_test)
predicted = np.argmax(predictions, axis = 1)
Ycat_test=np.argmax(Ycat_test, axis=1)

In [ ]:
print(pd.crosstab(Ycat_test, predicted))
ax = sns.heatmap(confusion_matrix(Ycat_test, predicted), annot = True, vmin = 0, vmax = 2000,fmt = 'g',cmap = 'Blues')

In [ ]:
print("Accuracy score:",round(accuracy_score(Ycat_test, predicted),4))
print("Number of correctly classified samples:",accuracy_score(Ycat_test, predicted, normalize=False))
print(classification_report(Ycat_test, predicted))